# Federated Tensorflow Mnist Tutorial With Dummy Envoy Shard Descriptor

This tutorial is based on the "Tensorflow_MNIST" tutorial but with changes to demostrate how to use the dummy shard descriptor. By default, OpenFL federation created by FedLCM will only use this dummy shard descriptor. And user have to specify how envoy can retreive local data by defining the `DataInterface`.

**Understanding of the original "Tensorflow_MNIST" is highly recommended!**

In [ ]:
# Install dependencies if not already installed
!python -m pip install tensorflow==2.8

## Imports

In [ ]:
import tensorflow as tf
print('TensorFlow', tf.__version__)

## Connect to the Federation

This cell connects this notebook to the Federation.

Note *the parameters provided in the cell is for the Jupyter Lab instance deployed along with the director by FedLCM. Change the parameters if necessary if you are running this notebook in other environments.*

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

client_id = 'api'
director_node_fqdn = 'director'
director_port = 50051
cert_chain = '/openfl/workspace/cert/root_ca.crt'
api_cert = '/openfl/workspace/cert/notebook.crt'
api_private_key = '/openfl/workspace/cert/priv.key'

federation = Federation(
  client_id=client_id,
  director_node_fqdn=director_node_fqdn,
  director_port=director_port,
  cert_chain=cert_chain,
  api_cert=api_cert,
  api_private_key=api_private_key
)

## Query Datasets from Shard Registry

In [ ]:
shard_registry = federation.get_shard_registry()
shard_registry

In [ ]:
# First, request a dummy_shard_desc that holds information about the federated dataset 
# If the shard descriptor is the default dummy one, the shape of the sample and target will be "1", which is not the actual data shape
dummy_shard_desc = federation.get_dummy_shard_descriptor(size=10)
dummy_shard_dataset = dummy_shard_desc.get_dataset('train')
sample, target = dummy_shard_dataset[0]
f"Sample shape: {sample.shape}, target shape: {target.shape}"

## Describing FL experimen

In [ ]:
from openfl.interface.interactive_api.experiment import TaskInterface
from openfl.interface.interactive_api.experiment import ModelInterface
from openfl.interface.interactive_api.experiment import FLExperiment

### Register model

In [ ]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=None),
], name='simplecnn')
model.summary()

# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=1e-3)

# Loss and metrics. These will be used later.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Create ModelInterface
framework_adapter = 'openfl.plugins.frameworks_adapters.keras_adapter.FrameworkAdapterPlugin'
MI = ModelInterface(model=model, optimizer=optimizer, framework_plugin=framework_adapter)

### Register dataset

This is the main difference between this tutorial and the original "Tensorflow_MNIST" tutorial. Instead of configuring each Envoy with a specific shard descriptor class, the dummy one is configuraed for them, and we move the data retrieval logic into this subclass from `DataInterface`. For other tasks not using the MNIST dataset, we can write our own local data retrival logic here.

In [ ]:
import os
import requests
import numpy as np
from tensorflow.keras.utils import Sequence
from openfl.interface.interactive_api.shard_descriptor import ShardDataset

from openfl.interface.interactive_api.experiment import DataInterface

class MnistShardDataset(ShardDataset):
    """Mnist Shard dataset class."""

    def __init__(self, x, y, rank=1, worldsize=1):
        """Initialize TinyImageNetDataset."""
        self.rank = rank
        self.worldsize = worldsize
        self.x = x[self.rank - 1::self.worldsize]
        self.y = y[self.rank - 1::self.worldsize]

    def __getitem__(self, index: int):
        """Return an item by the index."""
        return self.x[index], self.y[index]

    def __len__(self):
        """Return the len of the dataset."""
        return len(self.x)
    

class DataGenerator(Sequence):

    def __init__(self, data_set, batch_size):
        self.data_set = data_set
        self.batch_size = batch_size
        self.indices = np.arange(len(data_set))
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]

        X, y = self.data_set[batch]
        return X, y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)


class MnistFedDataset(DataInterface):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def shard_descriptor(self):
        return self._shard_descriptor

    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        
        (x_train, y_train), (x_test, y_test) = self.download_data()
        self.train_set = MnistShardDataset(x_train, y_train)
        self.valid_set = MnistShardDataset(x_test, y_test)
        
    def __getitem__(self, index):
        return self.shard_descriptor[index]

    def __len__(self):
        return len(self.shard_descriptor)
    
    def get_train_loader(self):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        if self.kwargs['train_bs']:
            batch_size = self.kwargs['train_bs']
        else:
            batch_size = 32
        return DataGenerator(self.train_set, batch_size=batch_size)

    def get_valid_loader(self):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        if self.kwargs['valid_bs']:
            batch_size = self.kwargs['valid_bs']
        else:
            batch_size = 32
        
        return DataGenerator(self.valid_set, batch_size=batch_size)

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_set)
    
    def download_data(self):
        """Download prepared dataset."""
        local_file_path = 'mnist.npz'
        mnist_url = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
        response = requests.get(mnist_url)
        with open(local_file_path, 'wb') as f:
            f.write(response.content)

        with np.load(local_file_path) as f:
            x_train, y_train = f['x_train'], f['y_train']
            x_test, y_test = f['x_test'], f['y_test']
            x_train = np.reshape(x_train, (-1, 28, 28, 1))
            x_test = np.reshape(x_test, (-1, 28, 28, 1))

        os.remove(local_file_path)  # remove mnist.npz
        print('Mnist data was loaded!')
        return (x_train, y_train), (x_test, y_test)

### Create Mnist federated dataset

In [ ]:
fed_dataset = MnistFedDataset(train_bs=64, valid_bs=512)

## Define and register FL tasks

In [ ]:
import time



TI = TaskInterface()

# from openfl.interface.aggregation_functions import AdagradAdaptiveAggregation    # Uncomment this lines to use 
# agg_fn = AdagradAdaptiveAggregation(model_interface=MI, learning_rate=0.4)       # Adaptive Federated Optimization
# @TI.set_aggregation_function(agg_fn)                                             # alghorithm!
#                                                                                  # See details in the:
#                                                                                  # https://arxiv.org/abs/2003.00295

@TI.register_fl_task(model='model', data_loader='train_dataset', device='device', optimizer='optimizer')     
def train(model, train_dataset, optimizer, device, loss_fn=loss_fn, warmup=False):
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * 64))
        if warmup:
            break

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

        
    return {'train_acc': train_acc,}


@TI.register_fl_task(model='model', data_loader='val_dataset', device='device')     
def validate(model, val_dataset, device):
    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
            
    return {'validation_accuracy': val_acc,}

## Time to start a federated learning experiment

In [ ]:
# create an experimnet in federation
experiment_name = 'mnist_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [ ]:
# print the default federated learning plan
import openfl.native as fx
print(fx.get_plan(fl_plan=fl_experiment.plan))

In [ ]:
# The following command zips the workspace and python requirements to be transfered to collaborator nodes
fl_experiment.start(model_provider=MI, 
                   task_keeper=TI,
                   data_loader=fed_dataset,
                   rounds_to_train=5,
                   opt_treatment='CONTINUE_GLOBAL',
                   override_config={'aggregator.settings.db_store_rounds': 1, 'compression_pipeline.template': 'openfl.pipelines.KCPipeline', 'compression_pipeline.settings.n_clusters': 2})

In [ ]:
fl_experiment.stream_metrics()